# Mode Selection
Set finetuning = 1 if you want to finetune, or set finetuning = 0 if you want to just load the saved model to get inference.

In [3]:
finetuning = 1
inference = not finetuning

# Drive Mount

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# Read Data

In [4]:
import pandas as pd
data = pd.read_csv('/kaggle/input/gb-all-necessary/train_test_toxicity_data.csv')
data

,male,female,_original_label,test
0,চোদনার পুত বদনার নাতি।। সালা জাত ফকিন্নি।। মাদ...,চোদনার মেয়া বদনার নাতনি । । সালী জাত ফকির । ...,1,0
1,এই জারজের বাচ্চা পত্রিকা সময়ের কন্ঠষ্যর যখন ভা...,এই জারজের বাচ্চা পত্রিকা সময়ের কন্ঠষ্যর যখন ভা...,1,0
2,"তোদের মত সবাই হাবলা পাঠা না , তোদের কি কোন কাজ...","তোদের মত সবাই হাবলা পাঁঠি না , তোদের কি কোন ...",1,0
3,"তোদের মত সবাই হাবলা পাঠা না , তোদের কি কোন কাজ...","তোদের মত সবাই হাবলা পাঁঠি না , তোদের কি কোন ...",1,0
4,যদি বদরুল শিবিরের নেতা হয় তাহলে ফাসি দেওয়া হক ...,যদি বদরুল শিব িরের নেত্রী হয় তাহলে ফাসি দেওয়া...,1,0
...,...,...,...,...
7047,নিশো ভাই তুমি সেরাদারুন নাটক,অমৃতা আফা তুমি সেরাদারুন নাটক,0,0
7048,নিশো ভাই তুমি সেরাদারুন নাটক,সাদিয়া বোন তুমি সেরাদারুন নাটক,0,1
7049,নিশো ভাই তুমি সেরাদারুন নাটক,দুর্গা ভাবী তুমি সেরাদারুন নাটক,0,1
7050,সত্যি বলতে কি এই রকম পাগল টাইপের ভালোবাসা আমার...,সত্যি বলতে কি এই রকম পাগলী টাইপের ভালোবাসা আমা...,0,0


# Train, Test, Validation Split
We are going to consider the data as test data if the prediction between original sentence and NER converted sentence become unequal.

In [5]:
temp = data[data['test']!= 1]
test = data[data['test']== 1]

In [7]:
len(temp), len(test)

(5642, 1410)

In [8]:
t1 = temp[['male', '_original_label']]
t2 = temp[['female', '_original_label']]
t2.columns = ['male', '_original_label']
temp = pd.concat([t1, t2], axis = 0)
temp

,male,_original_label
0,চোদনার পুত বদনার নাতি।। সালা জাত ফকিন্নি।। মাদ...,1
1,এই জারজের বাচ্চা পত্রিকা সময়ের কন্ঠষ্যর যখন ভা...,1
2,"তোদের মত সবাই হাবলা পাঠা না , তোদের কি কোন কাজ...",1
3,"তোদের মত সবাই হাবলা পাঠা না , তোদের কি কোন কাজ...",1
4,যদি বদরুল শিবিরের নেতা হয় তাহলে ফাসি দেওয়া হক ...,1
...,...,...
7045,ভালো লাগছে নাটকটা কিন্তু ১০ বছর বয়সে মেয়ে কলেজ...,0
7046,অসম্ভব সুন্দর এত ভালো লেগেছে নাটকটা অনেকদিন পর...,0
7047,অমৃতা আফা তুমি সেরাদারুন নাটক,0
7050,সত্যি বলতে কি এই রকম পাগলী টাইপের ভালোবাসা আমা...,0


In [9]:
validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=101))
train = temp.drop(validation.index)

<ipython-input-9-cf4ecadbd9c2>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  validation = temp.groupby('_original_label', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=101))


In [10]:
train

,male,_original_label
3,"তোদের মত সবাই হাবলা পাঠা না , তোদের কি কোন কাজ...",1
4,যদি বদরুল শিবিরের নেতা হয় তাহলে ফাসি দেওয়া হক ...,1
5,ছোটকা কী পারবে শ্রেষ্ঠা কে তুলে নিয়ে গিয়ে মাঝ ...,1
8,চুদির ভাই বাংলায় লিখতে পারো নাই,1
9,চুদির ভাই বাংলায় লিখতে পারো নাই,1
...,...,...
7044,অসাধারণ ভাল লাগল ড্রামাটা ধন্যবাদ পরিচালিকা ও ...,0
7045,ভালো লাগছে নাটকটা কিন্তু ১০ বছর বয়সে মেয়ে কলেজ...,0
7047,অমৃতা আফা তুমি সেরাদারুন নাটক,0
7050,সত্যি বলতে কি এই রকম পাগলী টাইপের ভালোবাসা আমা...,0


In [11]:
validation

,male,_original_label
6339,সত্যি ভাল লাগছে ভালবাসলে ভালবাসাকে রেস্পেক্ট ক...,0
3737,“আলাদা আলাদা ছেলে দিয়েছে”,0
5614,অসাধারন একটা নাটকইউনিটের সবাইকে অনেক ধন্যবাদ আ...,0
5673,আব্বাকে অফিসে নামিয়ে দিয়ে খালি গাড়ি নিয়ে বাসায়...,0
4446,অসাধারণ গল্প চমৎকার অভিনয় আর শামীম ভাই আমার প্...,0
...,...,...
3285,চিকিৎসক-নার্স-কর্মীদের সুরক্ষা উপকরণ সরবরাহের ...,1
1509,হিন্দুরা ব্রিটিশদের আর্য মনে করে পূন্য লাভের আ...,1
1771,"শালা শুয়োরের বাচ্চার অবস্থা দেখ, খানকির পুত জ...",1
810,ঐ শালী মালাউন । তোদের মা ইউরোপ আমেরিকাতে ঈদের ...,1


In [12]:
test

,male,female,_original_label,test
6,"কি দরকার ছিল এই ভিডিও বানানোর,পেটের দায়ে শরীর ...","কি দরকার ছিল এই ভিডিও বানানোর , পেটের দায়ে শরী...",1,1
7,চুদির ভাই বাংলায় লিখতে পারো নাই,চুদির আফা বাংলায় লিখতে পারো নাই,1,1
10,"যখন কেউ বলে অস্ত্রের চাইতে কলম বড়, তখন অস্ত্রধ...","যখন কেউ বলে অস্ত্রের চাইতে কলম বড় , তখন অস্ত্...",1,1
14,পরে দেখা গেলো বললেন বাঙ্গির ব্যাবসা । আপনারা ম...,পরে দেখা গেলো বললেন বাঙ্গির ব্যাবসা । আপনারা...,1,1
16,নাস্তিকদের বর্তমানের অন্যতম পুঁজি কোরানের একটি...,নাস্তিকদের বর্তমানের অন্যতম পুঁজি কোরানের একটি...,1,1
...,...,...,...,...
7032,নাটকটা সত্যিই অনেক অসাধারণ হয়েছে কিন্তু এখানে...,নাটকটা সত্যিই অনেক অসাধারণ হয়েছে কিন্তু এখানে...,0,1
7033,তার লাহোর প্রস্তাবেই ছিল স্বাধীন বাংলাদেশের বী...,তার লাহোর প্রস্তাবেই ছিল স্বাধীন বাংলাদেশের বী...,0,1
7037,নায়ক হিসেবে তাহসান ভাই থাকলে ভালো হতে,নায়িকা হিসেবে বিনিতা আফা থাকলে ভালো হতে,0,1
7048,নিশো ভাই তুমি সেরাদারুন নাটক,সাদিয়া বোন তুমি সেরাদারুন নাটক,0,1


# Load Pretrained Model

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transform
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertConfig

def load_transformer_based_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForMaskedLM.from_pretrained(model_path ,output_hidden_states=True)
    return tokenizer, model
model_path = "FredZhang7/one-for-all-toxicity-v3"
tokenizer, b_model = load_transformer_based_model(model_path)
print('Total Pretrained tokens: ',len(tokenizer.get_vocab()))
tokenizer.add_tokens(['<Name>' ,'<Gender>'])
b_model.resize_token_embeddings(len(tokenizer))
print('Total number of tokens after adding the new ones: ',len(tokenizer.get_vocab()))

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:818: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output

Total Pretrained tokens:  119547


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Total number of tokens after adding the new ones:  119549


# Dataset Preparation

In [14]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm



class BertDataset(Dataset):
    def __init__(self, data, tokenizer, train , gender, max_length):
        super(BertDataset, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length
        self.train = train
        self.gender = gender

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        if self.train == 1:
            text1 = self.train_csv.iloc[index,0] #0th column all data
        else:
            if self.gender == 'male':
                text1 = self.train_csv.iloc[index,0] #male
            else:
                text1 = self.train_csv.iloc[index,1] #female
                
                

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        
        try:
            if self.train == 1:
              return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 1], dtype=torch.long)
            else:
              return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 2], dtype=torch.long)
                
        except:
          print(index, self.train_csv[index])

dataset_train= BertDataset(train, tokenizer, train = 1 , gender = None, max_length = 512)
dataset_val= BertDataset( validation, tokenizer, train = 1 , gender = None, max_length = 512)
dataset_test_male = BertDataset(test, tokenizer, train = 0 , gender = 'male', max_length = 512)
dataset_test_female = BertDataset(test, tokenizer, train = 0 , gender = 'female', max_length = 512)

train_loader=DataLoader(dataset=dataset_train,batch_size=16, shuffle=True)
validation_loader=DataLoader(dataset=dataset_val,batch_size=16, shuffle=False)
test_loader_male=DataLoader(dataset=dataset_test_male,batch_size=16, shuffle=False)
test_loader_female=DataLoader(dataset=dataset_test_female,batch_size=16, shuffle=False)

In [13]:
# id,am, tid, l = next(iter(train_loader))
# x = b_model(id, am,tid)
# x.hidden_states[0].shape

# Model

In [15]:

class BERT(nn.Module):
    def __init__(self, model):
        super(BERT, self).__init__()
        self.bert_model = model
        self.dropout = nn.Dropout(0.2)
        self.out = nn.Linear(768, 1)

    def forward(self,ids,mask,token_type_ids):
        o2= self.bert_model(ids,mask,token_type_ids).hidden_states[0]
        o2= o2.max(dim=1).values
        o2 = self.dropout(o2)
        # print('------->>>>>>>>>',o2.values.shape)
        out= self.out(o2)

        return out, o2

masked_model=BERT(b_model)
# loss_fn = nn.BCEWithLogitsLoss()
# #Initialize Optimizer
# optimizer= optim.Adam(model.parameters(),lr= 0.0001)

In [15]:
# id,am, tid, l = next(iter(train_loader))
# x,_ = masked_model(id, am,tid)
# x.shape

# Training Loop Decorator

In [16]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product
import json
from torch.utils.tensorboard import SummaryWriter
import time
import torchvision
import pandas as pd
import torch
from IPython.display import display
from IPython.display import clear_output

class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple('Run', params.keys())


        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))


        return runs


class RunManager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None

        self.epoch_valid_loss = 0.0
        self.epoch_num_valid_correct = 0

        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None

        self.network = None
        self.loader = None
        self.validation_loader = None
        self.tb = None

    def begin_run(self, run, network, loader, validation_loader):
        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1

        self.network = network
        self.loader = loader
        self.validation_loader = validation_loader
        self.tb = SummaryWriter(comment=f'-{run}')



    def end_run(self):
        self.tb.close()
        self.epoch_count = 0

    def begin_epoch(self):
        self.epoch_start_time = time.time()
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_valid_loss = 0
        self.epoch_num_valid_correct = 0

    def end_epoch(self):
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        loss = self.epoch_loss/len(self.loader.dataset)
        accuracy = self.epoch_num_correct/len(self.loader.dataset)
        val_loss = self.epoch_valid_loss/len(self.validation_loader.dataset)
        val_accuracy = self.epoch_num_valid_correct/len(self.validation_loader.dataset)

        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        self.tb.add_scalar('Validation Loss', val_loss, self.epoch_count)
        self.tb.add_scalar('Validation Accuracy', val_accuracy, self.epoch_count)

        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)

        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["validation loss"] = val_loss
        results["validation accuracy"] = val_accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')

        clear_output(wait=True)
        display(df)

    def track_loss(self, loss):
        self.epoch_loss += loss.item()#*self.loader.batch_size

    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)

    def track_validation_loss(self, loss):
        self.epoch_valid_loss += loss.item()#*self.loader.batch_size

    def track_num_validation_correct(self, preds, labels):
        self.epoch_num_valid_correct += self._get_num_correct(preds, labels)

    @torch.no_grad()
    def _get_num_correct(self, preds, labels):
        return torch.where(preds >= 0, 1, 0).eq(labels).sum().item()

    def save(self, fileName):
        pd.DataFrame.from_dict(
            self.run_data,
            orient='columns').to_csv(rf'{fileName}.csv')


# Training

In [17]:
SAVE_MODEL = True
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup, AdamW
def my_trainer(train_loader, validation_loader, network, LR = 1e-5 ,epochs = 5, BATCH_SIZE = 16,  fresh_training = True, validation = True):
    params = OrderedDict(
        lr = [LR],
        batch_size =[BATCH_SIZE],
        device = ['cuda' if torch.cuda.is_available() else 'cpu'],
    )

    m = RunManager()
    for run in RunBuilder.get_runs(params):
        device = torch.device(run.device)
        network = network.to(device)
        # optimizer = optim.Adam(network.parameters(), lr=run.lr)

        loss_fn = nn.BCEWithLogitsLoss()
        total_steps = len(train_loader) * epochs
        optimizer = optim.Adam(network.parameters(), lr= run.lr)

        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)
        if not fresh_training:
            load_model(network, optimizer, PATH = 'foa_toxicity.pth')
        m.begin_run(run, network, train_loader, validation_loader)
        for epoch in range(epochs):
            network.train(True)
            print(f'Epoch {epoch + 1} : ', end = '\t')
            m.begin_epoch()
            for batch in tqdm(train_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)

                preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                labels = labels.type_as(preds).view(-1, 1)
                # print(labels.shape)
                # print('----------')
                # print(preds.shape)


                loss = loss_fn(preds, labels)
                # print(preds.shape, labels.shape)
                # print(preds, labels)


                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(network.parameters(), 1.0)
                optimizer.step()
                scheduler.step()


                m.track_loss(loss)
                m.track_num_correct(preds, labels)


            network.eval()
            for batch in tqdm(validation_loader):
                input_ids, token_type_ids, attention_masks, labels = batch
                input_ids = input_ids.to(device)
                token_type_ids = token_type_ids.to(device)
                attention_masks = attention_masks.to(device)
                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)
                with torch.no_grad():
                    preds,hidden_states = network(input_ids, attention_masks,token_type_ids)
                    labels = labels.type_as(preds).view(-1, 1)
                    loss = loss_fn(preds, labels)
                    m.track_validation_loss(loss)
                    m.track_num_validation_correct(preds, labels)
                # break #not doing validation here




            m.end_epoch()
            network.train(False)
        m.end_run()
    if SAVE_MODEL == True:
        save_model(network, optimizer)

    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'results_{s}'
    m.save(PATH)
    return


def save_model(network, optimizer):
    s = datetime.now().strftime("%d-%m-%Y %I:%M:%p").replace(" ", "_").replace(":", "_")
    PATH  = rf'foa_toxicity.pth'
    torch.save({
    'model_state_dict': network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    }, PATH)
    print(f'Saved model and optimizer at {PATH}')
    return

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH)
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return

def get_parameter_info(network, details = False):
    print('Total parameters:' ,(sum(p.numel() for p in network.parameters())))
    print('Total trainable parameters:' ,(sum(p.numel() for p in network.parameters() if p.requires_grad)))
    if details:
        print('---------------------------------\nDetailed Parameter Info: ')

        for name, parameter in network.named_parameters():
            print('name, parameter.requires_grad, parameter.is_cuda, parameter.size(): ', name, parameter.requires_grad, parameter.is_cuda, parameter.size())
            print('---------------------------------')
    return


if finetuning:
  my_trainer(train_loader, validation_loader, masked_model, 0.0001, epochs = 15, BATCH_SIZE = 16, fresh_training = True)

,run,epoch,loss,accuracy,validation loss,validation accuracy,epoch duration,run duration,lr,batch_size,device
0,1,1,0.031553,0.753196,0.020498,0.880372,296.715224,296.717427,0.0001,16,cuda
1,1,2,0.015937,0.899389,0.014872,0.906513,296.336372,596.858000,0.0001,16,cuda
2,1,3,0.011209,0.932185,0.012743,0.917590,296.467427,896.964631,0.0001,16,cuda
3,1,4,0.008780,0.945942,0.011645,0.927337,296.391595,1197.024689,0.0001,16,cuda
4,1,5,0.007091,0.958310,0.010985,0.930882,296.441691,1497.082544,0.0001,16,cuda
5,1,6,0.005833,0.967204,0.010740,0.933983,296.320147,1797.113157,0.0001,16,cuda
6,1,7,0.004906,0.974013,0.010485,0.933540,296.427941,2097.123169,0.0001,16,cuda
7,1,8,0.004190,0.976376,0.010203,0.937528,296.884205,2397.646267,0.0001,16,cuda
8,1,9,0.003540,0.980545,0.010091,0.938414,296.824449,2698.080389,0.0001,16,cuda
9,1,10,0.003242,0.982768,0.010044,0.939300,296.915607,2998.621195,0.0001,16,cuda


Saved model and optimizer at foa_toxicity.pth


# Load Model

In [17]:
#Load Model
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict
from datetime import datetime
from transformers import get_linear_schedule_with_warmup, AdamW

def load_model(network, optimizer, PATH ):
    checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
    network.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print(rf'Loaded model and optimizer from {PATH}')
    else:
        print(rf'Loaded model from {PATH}')

    return


if inference:
  load_model(masked_model, None, PATH = '/kaggle/input/foa_toxicity/pytorch/default/1/foa_toxicity.pth')

<ipython-input-17-cca5971061c9>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(PATH, map_location=torch.device('cpu'))


Loaded model from /content/drive/MyDrive/#Research/# GB/saved_model/baseline_senti.pth


# Inference

In [18]:
def get_prediction(data_loader, network):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  device = torch.device(device)
  network = network.to(device)
  network.eval()
  l = []
  emb = []
  for batch in tqdm(data_loader):
    input_ids, token_type_ids, attention_masks, labels = batch
    input_ids = input_ids.to(device)
    token_type_ids = token_type_ids.to(device)
    attention_masks = attention_masks.to(device)
    labels = labels.type(torch.LongTensor)
    labels = labels.to(device)
    with torch.no_grad():
        preds, hidden_states = network(input_ids, attention_masks,token_type_ids)
        labels = labels.type_as(preds).view(-1, 1)
        preds =  torch.where(preds >= 0, 1, 0)
        l.append(preds)
        emb.append(hidden_states)
  return l, torch.vstack(emb).cpu().numpy()

import warnings 
warnings.filterwarnings('ignore') 

p, emb_male = get_prediction(test_loader_male, masked_model)
test['pred_male_fod'] = [x[0] for pred in p for x in pred.tolist()]
p, emb_male = get_prediction(test_loader_female, masked_model)
test['pred_female_fod'] = [x[0] for pred in p for x in pred.tolist()]
x = sum(test['pred_male_fod']!=test['_original_label'])
print('For Male:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test))) 
x = sum(test['pred_female_fod']!=test['_original_label'])
print('For Female:\nTotal test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test)))
print('Male Female Mismatch: ', sum(test['pred_male_fod']!=test['pred_female_fod']))

100%|██████████| 89/89 [00:41<00:00,  2.15it/s]

For Male:
Total test data:  1410 Total mismatch:  130 Accuracy:  0.9078014184397163
For Female:
Total test data:  1410 Total mismatch:  133 Accuracy:  0.9056737588652483
Male Female Mismatch:  59


# FOA Result

In [19]:
import numpy as np
np.random.seed(0)

def bootstrap_ci(y_true, y_pred, B=500):
    """Compute bootstrap confidence interval for accuracy."""
    N = len(y_true)
    original_acc = np.mean(y_pred == y_true)

    bootstrap_accs = []
    
    for _ in range(B):
        indices = np.random.choice(N, N, replace=True)  # Sample with replacement
        y_sample = y_true[indices]
        acc = np.mean(y_pred[indices] == y_sample)
        bootstrap_accs.append(acc)
    # Compute 95% confidence interval
    lower, upper = np.percentile(bootstrap_accs, [2.5, 97.5])
    
    return original_acc, (lower, upper), np.mean(bootstrap_accs)

y_true = test['_original_label'].values  # Ground truth labels
y_pred = test['pred_male_fod'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('Male: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")

y_true = test['_original_label'].values
y_pred = test['pred_female_fod'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('\nFemale: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")
test.to_csv('2,3.revision_foa_toxicity_result.csv', index = False)

Male: 
Accuracy: 0.9078
95% Confidence Interval: (0.8929078014184397, 0.9219858156028369)
mean accuracy: 0.9072723404255318

Female: 
Accuracy: 0.9057
95% Confidence Interval: (0.8921985815602836, 0.9198581560283688)
mean accuracy: 0.9056553191489362


In [1]:
# Statistical Parity Difference (SPD) and Equal Opportunity Difference (EOD) Calculation
import numpy as np
import pandas as pd
np.random.seed(0)

test = pd.read_csv('revision_foa_toxicity_result.csv')


# Function to calculate Statistical Parity Difference (SPD)
def calculate_spd(male_pred, female_pred):
    # Calculate probabilities of positive predictions for males and females
    p_male = np.mean(male_pred == 1)  # Proportion of positive predictions for males
    p_female = np.mean(female_pred == 1)  # Proportion of positive predictions for females
    
    # Return the absolute value of the Statistical Parity Difference
    return np.abs(p_male - p_female)

# Function to calculate Equal Opportunity Difference (EOD)
def calculate_eod(male_pred, female_pred, y_true):
    # True Positive Rate for males
    tpr_male = np.mean((male_pred == 1) & (y_true == 1))  # True positives for males
    
    # True Positive Rate for females
    tpr_female = np.mean((female_pred == 1) & (y_true == 1))  # True positives for females
    
    # Return the absolute value of the Equal Opportunity Difference
    return np.abs(tpr_male - tpr_female)

# Function to calculate bootstrap confidence intervals for SPD and EOD
def bootstrap_ci_spd_eod(male_pred, female_pred, y_true, n_iterations=500, ci=95):
    n = len(male_pred)
    
    # Arrays to store SPD and EOD for each bootstrap sample
    spd_values = np.zeros(n_iterations)
    eod_values = np.zeros(n_iterations)
    
    # Perform bootstrap sampling
    for i in range(n_iterations):
        # Resample with replacement
        sample_indices = np.random.choice(n, size=n, replace=True)
        male_pred_resampled = male_pred[sample_indices]
        female_pred_resampled = female_pred[sample_indices]
        y_true_resampled = y_true[sample_indices]
        
        # Calculate SPD and EOD for the resampled data
        spd_values[i] = calculate_spd(male_pred_resampled, female_pred_resampled)
        eod_values[i] = calculate_eod(male_pred_resampled, female_pred_resampled, y_true_resampled)
    
    # Calculate the mean of SPD and EOD
    spd_mean = np.mean(spd_values)
    eod_mean = np.mean(eod_values)
    
    # Calculate the confidence interval bounds for SPD and EOD
    lower_percentile = (100 - ci) / 2
    upper_percentile = 100 - lower_percentile
    spd_lower_bound = np.percentile(spd_values, lower_percentile)
    spd_upper_bound = np.percentile(spd_values, upper_percentile)
    eod_lower_bound = np.percentile(eod_values, lower_percentile)
    eod_upper_bound = np.percentile(eod_values, upper_percentile)
    
    return (np.abs(spd_mean), np.abs(spd_lower_bound), np.abs(spd_upper_bound)), (np.abs(eod_mean), np.abs(eod_lower_bound), np.abs(eod_upper_bound))


y_true = test._original_label.values
male_pred = test.pred_male_fod.values
female_pred = test.pred_female_fod.values

# Calculate bootstrap CI and mean for SPD and EOD
(spd_mean, spd_lower, spd_upper), (eod_mean, eod_lower, eod_upper) = bootstrap_ci_spd_eod(
    male_pred, female_pred, y_true, n_iterations=500, ci=95
)

print(f"SPD 95% Confidence Interval: [{spd_lower:.3f}, {spd_upper:.3f}]\tSPD Mean: {spd_mean:.3f}")
print('---------------------------------------------------------------------------------------------')
print(f"EOD 95% Confidence Interval: [{eod_lower:.3f}, {eod_upper:.3f}]\tEOD Mean: {eod_mean:.3f}")

c:\Users\joyau\anaconda3\envs\pytorch_gpu\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


SPD 95% Confidence Interval: [0.000, 0.014]	SPD Mean: 0.005
---------------------------------------------------------------------------------------------
EOD 95% Confidence Interval: [0.000, 0.011]	EOD Mean: 0.004


# Mismatched Samples

In [20]:
test[test['pred_male_fod']!=test['pred_female_fod']]

,male,female,_original_label,test,pred_male_fod,pred_female_fod
211,"লোক মুখে শুনতাম, ভিখারি ধনি হলে, অভ্যাসটা ভিখা...","স্ত্রীলোক মুখে শুনতাম , ভিখারি ধনি হলে , অভ্...",1,1,1,0
212,"লোক মুখে শুনতাম, ভিখারি ধনি হলে, অভ্যাসটা ভিখা...","মহিলা মুখে শুনতাম , ভিখারি ধনি হলে , অভ্যাসট...",1,1,1,0
230,আফাঢারে কেউ বুঝাও যে তার চেহারা দেখে তার জামাই...,ভাইঢারে কেউ বুঝাও যে তার চেহারা দেখে তার বউ ভয...,1,1,0,1
790,ভাই রানু মণ্ডল হলো অকৃতজ্ঞ এক কথায়,বোন আদৃতা হলো অকৃতজ্ঞ এক কথায়,1,1,1,0
867,ফালতু নাঁচ দেখিয়ে ফলোয়ার বাড়ানো লোক হলেন ইনি,ফালতু নাঁচ দেখিয়ে ফলোয়ার বাড়ানো স্ত্রীলোক হলেন...,1,1,1,0
1213,নূর ভাই আপনি আল্লাহর ওপর ভরসা রাখুন সদা সর্বদা...,আয়েশা বোন আপনি আল্লাহর ওপর ভরসা রাখুন সদা সর...,1,1,1,0
1252,শ্রেষ্ঠ গর্দভ,শ্রেষ্ঠা গর্দভী,1,1,0,1
1286,আমি ও রানু দিদি কে সাপোর্ট করতাম কিন্তু আজ আমি...,আমি ও অর্ক দাদা কে সাপোর্ট করতাম কিন্তু আজ আ...,1,1,1,0
1374,মেয়ের বয়স মাএ ১৫ বছরএর মধ্যেই যৌবনের জ্বালা ধর...,ছেলের বয়স মাএ ১৫ বছরএর মধ্যেই যৌবনের জ্বালা ধর...,1,1,0,1
1422,এই লোক গুলো এমন ভাবে লেখেছে যেন মনেহচ্ছে মায়ের...,এই নারী গুলো এমন ভাবে লেখেছে যেন মনেহচ্ছে মার ...,1,1,0,1


In [20]:
# After Finetuning
x = sum(test['original_lebel'] != test['pred'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)) )

Total test data:  218 Total mismatch:  26 Accuracy:  0.8807339449541285


In [ ]:
# After Finetuning
x = sum(test['original_lebel'] != test['pred'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)) )

Total test data:  218 Total mismatch:  26 Accuracy:  0.8807339449541285


In [ ]:
# Before fine tuning
sum(data['Predicted Value for Original Sentence'] != data['Predicted Value for ner converted'])

218


# Prediction of NER Converted text data

In [21]:
import pandas as pd
import numpy as np
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm


class BertDatasetNER(Dataset):
    def __init__(self, data, tokenizer,max_length):
        super(BertDatasetNER, self).__init__()

        self.train_csv= data
        self.tokenizer=tokenizer
        self.target=self.train_csv.iloc[:,1]
        self.max_length=max_length

    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):

        text1 = self.train_csv.iloc[index,1] #1 column for NER Converted

        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            truncation = True
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        try:
          return torch.tensor(ids, dtype=torch.long), torch.tensor(token_type_ids, dtype=torch.long), torch.tensor(mask, dtype=torch.long), torch.tensor(self.train_csv.iloc[index, 3], dtype=torch.long)
        except:
          print(index, self.train_csv[index])



dataset_test= BertDatasetNER(test, tokenizer, max_length=200)
test_loader=DataLoader(dataset=dataset_test,batch_size=16, shuffle = False)
p, emb = get_prediction(test_loader, masked_model)
test['pred_ner'] = [x[0] for pred in p for x in pred.tolist()]
x = sum(test['original_lebel'] != test['pred_ner'])
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

100%|██████████| 14/14 [00:02<00:00,  4.99it/s]

Total test data:  218 Total mismatch:  35 Accuracy:  0.8394495412844036 Bias:  17


In [22]:
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

Total test data:  218 Total mismatch:  35 Accuracy:  0.8394495412844036 Bias:  17


In [ ]:
print('Total test data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1 - (x/len(test)), 'Bias: ', sum(test['pred'] != test['pred_ner']) )

Total test data:  218 Total mismatch:  36 Accuracy:  0.8348623853211009 Bias:  18


In [ ]:
test.to_csv('/content/drive/MyDrive/#Research/# GB/baseline_senti_reSult.csv', index = False)